In [1]:
!git clone https://github.com/HardikMochi/Assignment.git

Cloning into 'Assignment'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 3), reused 12 (delta 1), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [2]:
cd Assignment

/content/Assignment


In [3]:
!pip install streamlit

     |████████████████████████████████| 8.2MB 3.7MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 112kB 42.9MB/s 
     |████████████████████████████████| 4.2MB 24.6MB/s 
     |████████████████████████████████| 163kB 40.6MB/s 
     |████████████████████████████████| 122kB 41.3MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=cea73921f74acb64f03a1b6b8474930c39243d77f99e1ee923b7bf38904959fb
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [4]:
!pip install faiss-cpu
!pip install -U sentence-transformers

     |████████████████████████████████| 8.2MB 4.7MB/s 
     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.1MB 8.9MB/s 
     |████████████████████████████████| 1.2MB 50.0MB/s 
     |████████████████████████████████| 901kB 41.7MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=ce55be296fda0c138228c202f2709ff047f989ba026cac1f67eb7e909cb2afc0
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


In [5]:

%%writefile app.py
  
import faiss
import pickle
import pandas as pd
import streamlit as st
from sentence_transformers import SentenceTransformer
from utils import vector_search,id2details
import numpy as np


@st.cache
def read_data(data="data/misinformation_papers.csv"):
    """Read the data from local."""
    return pd.read_csv(data)


@st.cache(allow_output_mutation=True)
def load_bert_model(name="distilbert-base-nli-stsb-mean-tokens"):
    """Instantiate a sentence-level DistilBERT model."""
    return SentenceTransformer(name)

@st.cache(allow_output_mutation=True)
def load_faiss_index(path_to_faiss="models/faiss_index.pickle"):
    """Load and deserialize the Faiss index."""
    with open(path_to_faiss, "rb") as h:
        data = pickle.load(h)
    return faiss.deserialize_index(data)


def main():
    # Load data and models
    data = read_data()
    model = load_bert_model()
    faiss_index = load_faiss_index()

    st.title("Top 5 ranked articles for that query")

    # User search
    user_input = st.text_area("Search box", "Understanding the perception of COVID-19 policies by mining a multilanguage Twitter dataset")

    # Fetch results
    if user_input:
        # Get paper IDs
        D, I = vector_search([user_input], model, faiss_index, num_results=5)
        titles = np.array(id2details(data, I, 'original_title'))
        abstarct = np.array(id2details(data, I, 'abstract'))
        # Get individual results
        for i in range(5):
            st.write(
                f"""**title :{titles[i]}**   
            **Abstract :**
            {abstarct[i]}
            """
            )


if __name__  == "__main__":
    main()



Writing app.py


In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2021-04-25 04:59:02--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.242.214, 3.224.116.172, 35.170.116.11, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.242.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13828408 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  48.2MB/s    in 0.3s    

2021-04-25 04:59:02 (48.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13828408/13828408]



In [8]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0587fecb3602.ngrok.io


In [9]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.231.5.11:8501

2021-04-25 04:59:16.193 Loading faiss with AVX2 support.
2021-04-25 04:59:16.194 Loading faiss.
2021-04-25 04:59:21.446762: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-25 04:59:23.414 Load pretrained SentenceTransformer: distilbert-base-nli-stsb-mean-tokens
2021-04-25 04:59:23.415 Did not find folder distilbert-base-nli-stsb-mean-tokens
2021-04-25 04:59:23.415 Search model on server: http://sbert.net/models/distilbert-base-nli-stsb-mean-tokens.zip
2021-04-25 04:59:23.416 Downloading sentence transformer model from http://sbert.net/models/distilbert-base-nli-stsb-mean-tokens.zip and saving it at /root/.cache/torch/sentence_transformers/sbert.net_models_distilbert-base-nli-stsb-mean-tokens
100% 245M/245M [00:13<00:00, 18.7MB/s]
2021-04-25 04:59:40.436 Load SentenceTransform